In [1]:
# load Preliminaries
from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json, random
import networkx as nx
import os
from glob import glob

In [2]:
###### Parameters #######
DATASETS = [{'name': 'letter', 'threshold': 1},
            {'name': 'AIDS', 'threshold': 1},
            {'name': 'mutagenicity', 'threshold': 5}]
idx = 1
dataset = DATASETS[idx]['name']
threshold = DATASETS[idx]['threshold']
folder = os.path.join('../results/draw/', dataset, '')
experiment = 'pagerank'

In [3]:
###### Data ######
graphs = []
print
for filename in glob(f'{folder}*.json'):
    if not filename.split('/')[-1].startswith(experiment):
        continue
    print(filename)
    with open(filename, mode='r') as fp:
        tmp_graph = json.load(fp)
        graphs.append(tmp_graph)


# print(*graphs, sep='\n\n')

../results/draw/AIDS/pagerank_molid646232.json
../results/draw/AIDS/pagerank_molid684035.json
../results/draw/AIDS/pagerank_molid634193.json
../results/draw/AIDS/pagerank_molid646645.json
../results/draw/AIDS/pagerank_molid686667.json
../results/draw/AIDS/pagerank_molid669986.json
../results/draw/AIDS/pagerank_molid663915.json
../results/draw/AIDS/pagerank_molid680831.json
../results/draw/AIDS/pagerank_molid30152.json
../results/draw/AIDS/pagerank_molid369774.json


In [4]:
# loading the engines locally along with plug-in.
HTML('''
<script src="./sigma.min.js"></script>
''')

In [5]:
if dataset == 'mutagenicity':
    for graph in graphs:
        nx_graph = nx.Graph()
        for node in graph['nodes']:
            nx_graph.add_node(node['id'])
    
        for edge in graph['edges']:
            nx_graph.add_edge(edge['source'], edge['target'], weight=int(edge['label']))
    
        # pos = nx.drawing.layout.spring_layout(nx_graph, iterations=2500, center=[0.0, 0.0], seed=42)
        pos = nx.drawing.layout.kamada_kawai_layout(nx_graph)

        for node in graph['nodes']:
            node['x'], node['y'] = pos[node['id']]

In [22]:
js_text_template = Template('''

    var g = $graph_data ;

s = new sigma({graph: g, container: '$container', settings: { defaultNodeColor: '#ec5148', labelThreshold: $threshold, zoomingRatio:1.4} });

s.graph.nodes().forEach(function(n) {
  n.originalColor = n.color;
});
s.graph.edges().forEach(function(e) {
  e.originalColor = e.color;
});

s.bind('clickNode', function(e) {
  var nodeId = e.data.node.id,
      toKeep = s.graph.neighbors(nodeId);
  toKeep[nodeId] = e.data.node;

  s.graph.nodes().forEach(function(n) {
    if (toKeep[n.id])
      n.color = n.originalColor;
    else
      n.color = '#eee';
  });

  s.graph.edges().forEach(function(e) {
    if (toKeep[e.source] && toKeep[e.target])
      e.color = e.originalColor;
    else
      e.color = '#eee';
  });

  s.refresh();
});

s.bind('clickStage', function(e) {
  s.graph.nodes().forEach(function(n) {
    n.color = n.originalColor;
  });

  s.graph.edges().forEach(function(e) {
    e.color = e.originalColor;
  });

  s.refresh();
});

s.refresh();

// Listeners
var force = false;
document.getElementById('layout').onclick = function() {
  if (!force)
    s.startForceAtlas2({worker: true});
  else
    s.stopForceAtlas2();
  force = !force;
};

document.getElementById('export').onclick = function() {
  console.log('exporting...');
  var output = s.toSVG({download: true, filename: '$filename', size: 1000, labels: true});
  // console.log(output);
};

// Initialize the dragNodes plugin:
var dragListener = sigma.plugins.dragNodes(s, s.renderers[0]);

dragListener.bind('startdrag', function(event) {
  console.log(event);
});
dragListener.bind('drag', function(event) {
  console.log(event);
});
dragListener.bind('drop', function(event) {
  console.log(event);
});
dragListener.bind('dragend', function(event) {
  console.log(event);
});


''')

In [23]:
html_template = Template('''

<div id="graph-div" style="height:1000px"></div>
<button id="layout" type="button">Layout</button>
<button id="export" type="export">Export</button>
<script> $js_text </script>
''')

In [24]:
idx_graph = 5
graph_data = graphs[idx_graph]
js_text = js_text_template.substitute({'graph_data': json.dumps(graph_data),
                                       'container': 'graph-div',
                                       'threshold': threshold,
                                       'filename': 'test.svg'
                                      })
HTML(html_template.substitute({'js_text': js_text}))

In [25]:
general_template = Template('''
 <!DOCTYPE html>
<html>
<head>
<title>Page Title</title>
</head>
<body>
</body>
<script src="./sigma.js/src/sigma.core.js"></script>
<script src="./sigma.js/src/conrad.js"></script>
<script src="./sigma.js/src/utils/sigma.utils.js"></script>
<script src="./sigma.js/src/utils/sigma.polyfills.js"></script>
<script src="./sigma.js/src/sigma.settings.js"></script>
<script src="./sigma.js/src/classes/sigma.classes.dispatcher.js"></script>
<script src="./sigma.js/src/classes/sigma.classes.configurable.js"></script>
<script src="./sigma.js/src/classes/sigma.classes.graph.js"></script>
<script src="./sigma.js/src/classes/sigma.classes.camera.js"></script>
<script src="./sigma.js/src/classes/sigma.classes.quad.js"></script>
<script src="./sigma.js/src/classes/sigma.classes.edgequad.js"></script>
<script src="./sigma.js/src/captors/sigma.captors.mouse.js"></script>
<script src="./sigma.js/src/captors/sigma.captors.touch.js"></script>
<script src="./sigma.js/src/renderers/sigma.renderers.canvas.js"></script>
<script src="./sigma.js/src/renderers/sigma.renderers.webgl.js"></script>
<script src="./sigma.js/src/renderers/sigma.renderers.svg.js"></script>
<script src="./sigma.js/src/renderers/sigma.renderers.def.js"></script>
<script src="./sigma.js/src/renderers/webgl/sigma.webgl.nodes.def.js"></script>
<script src="./sigma.js/src/renderers/webgl/sigma.webgl.nodes.fast.js"></script>
<script src="./sigma.js/src/renderers/webgl/sigma.webgl.edges.def.js"></script>
<script src="./sigma.js/src/renderers/webgl/sigma.webgl.edges.fast.js"></script>
<script src="./sigma.js/src/renderers/webgl/sigma.webgl.edges.arrow.js"></script>
<script src="./sigma.js/src/renderers/canvas/sigma.canvas.labels.def.js"></script>
<script src="./sigma.js/src/renderers/canvas/sigma.canvas.hovers.def.js"></script>
<script src="./sigma.js/src/renderers/canvas/sigma.canvas.nodes.def.js"></script>
<script src="./sigma.js/src/renderers/canvas/sigma.canvas.edges.def.js"></script>
<script src="./sigma.js/src/renderers/canvas/sigma.canvas.edges.curve.js"></script>
<script src="./sigma.js/src/renderers/canvas/sigma.canvas.edges.arrow.js"></script>
<script src="./sigma.js/src/renderers/canvas/sigma.canvas.edges.curvedArrow.js"></script>
<script src="./sigma.js/src/renderers/canvas/sigma.canvas.edgehovers.def.js"></script>
<script src="./sigma.js/src/renderers/canvas/sigma.canvas.edgehovers.curve.js"></script>
<script src="./sigma.js/src/renderers/canvas/sigma.canvas.edgehovers.arrow.js"></script>
<script src="./sigma.js/src/renderers/canvas/sigma.canvas.edgehovers.curvedArrow.js"></script>
<script src="./sigma.js/src/renderers/canvas/sigma.canvas.extremities.def.js"></script>
<script src="./sigma.js/src/renderers/svg/sigma.svg.utils.js"></script>
<script src="./sigma.js/src/renderers/svg/sigma.svg.nodes.def.js"></script>
<script src="./sigma.js/src/renderers/svg/sigma.svg.edges.def.js"></script>
<script src="./sigma.js/src/renderers/svg/sigma.svg.edges.curve.js"></script>
<script src="./sigma.js/src/renderers/svg/sigma.svg.labels.def.js"></script>
<script src="./sigma.js/src/renderers/svg/sigma.svg.hovers.def.js"></script>
<script src="./sigma.js/src/middlewares/sigma.middlewares.rescale.js"></script>
<script src="./sigma.js/src/middlewares/sigma.middlewares.copy.js"></script>
<script src="./sigma.js/src/misc/sigma.misc.animation.js"></script>
<script src="./sigma.js/src/misc/sigma.misc.bindEvents.js"></script>
<script src="./sigma.js/src/misc/sigma.misc.bindDOMEvents.js"></script>
<script src="./sigma.js/src/misc/sigma.misc.drawHovers.js"></script>

<script src="./sigma.min.js"></script>
<script src="./sigma.js/plugins/sigma.layout.forceAtlas2/worker.js"></script>
<script src="./sigma.js/plugins/sigma.layout.forceAtlas2/supervisor.js"></script>
<script src="./sigma.js/plugins/sigma.exporters.svg/sigma.exporters.svg.js"></script>
<script src="./sigma.js/plugins/sigma.plugins.dragNodes/sigma.plugins.dragNodes.js"></script>

$test
</html>
''')

In [26]:

graph_data = graphs[0]
js_text = js_text_template.substitute({'graph_data': json.dumps(graph_data),
                                       'container': 'graph-div',
                                       'threshold': threshold
                                       'filename':
                                      })
s = html_template.substitute({'js_text': js_text})
val = general_template.substitute({'test': s})

KeyError: 'filename'

In [27]:
# print(val)
with open('test.html', mode='w') as file:
    file.write(val)